In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [31]:
np.random.randint(1, min((20 + 2) // 3, 8)) 

5

In [35]:
# Function to generate data
def generate_data(max_length, num_samples, p):
    data = []
    while len(data) < num_samples:
        n = np.random.randint(1, min((max_length + 2) // 3, 8))  # Limit n to ensure total length <= 20
        total_length = 3 * n
        if total_length > max_length:
            continue

        # Determine if the sample should be from the language or not based on probability p
        if np.random.rand() < p:
            sample = 'a' * n + 'b' * n + 'c' * n
            data.append((sample, 1))  # Label 1 for samples in the language
        else:
            sample = ''
            for _ in range(total_length):
                char = np.random.choice(['a', 'b', 'c'])
                sample += char
            data.append((sample, 0))  # Label 0 for non-language samples

    return data

In [38]:
generate_data(20, 20,0.5)

[('aaabbbccc', 1),
 ('bbb', 0),
 ('aaaaaabbbbbbcccccc', 1),
 ('bcbcbcabacbbcbcaaa', 0),
 ('aabcacccacbcbcbaaa', 0),
 ('aaaabbbbcccc', 1),
 ('acabbb', 0),
 ('acabbbccc', 0),
 ('aaaaaabbbbbbcccccc', 1),
 ('babbbcbcabacbbcaaa', 0),
 ('aaabbbccc', 1),
 ('baa', 0),
 ('aabbcc', 1),
 ('cacaca', 0),
 ('aaabbbccc', 1),
 ('aaaaabbbbbccccc', 1),
 ('acbabcacbabccccacc', 0),
 ('aabbcc', 1),
 ('aaaabbbbcccc', 1),
 ('aaaabbbbcccc', 1)]

In [39]:
# Function to convert data to tensors
def data_to_tensor(data):
    X = []
    y = []
    for sample, label in data:
        sample_tensor = torch.zeros(len(sample), dtype=torch.long)
        for i, char in enumerate(sample):
            if char == 'a':
                sample_tensor[i] = 0
            elif char == 'b':
                sample_tensor[i] = 1
            else:
                sample_tensor[i] = 2
        X.append(sample_tensor)
        y.append(label)

    return torch.stack(X), torch.tensor(y, dtype=torch.float)

In [ ]:
# Define the RNN model
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        _, hidden = self.rnn(x)
        output = self.fc(hidden.squeeze(0))
        return output

# Function to train the model


In [ ]:
def train_model(model, X_train, y_train, X_val, y_val, num_epochs=100, lr=0.001):
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()

        output = model(X_train)
        loss = criterion(output.squeeze(), y_train)
        loss.backward()
        optimizer.step()

        # Validation loss
        model.eval()
        with torch.no_grad():
            val_output = model(X_val)
            val_loss = criterion(val_output.squeeze(), y_val)

        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}')

In [ ]:


# Generate data
data = generate_data(max_length=20, num_samples=1000, p=0.8)  # 80% of the samples are from the language
X, y = data_to_tensor(data)

# Split data into training and validation sets
split_idx = int(0.8 * len(data))
X_train, X_val = X[:split_idx], X[split_idx:]
y_train, y_val = y[:split_idx], y[split_idx:]

# Initialize and train the RNN model
input_size = 3  # one-hot encoding for characters a, b, c
hidden_size = 16
output_size = 1
rnn_model = RNNModel(input_size, hidden_size, output_size)
train_model(rnn_model, X_train.unsqueeze(0), y_train, X_val.unsqueeze(0), y_val)